In [1]:
# python을 이용 hadoop file system을 다루는 모듈 패키지(원래 명령어로 하던 것들)
# !pip install hdfs
# ref : https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from hdfs import InsecureClient
import requests
import json
import datetime as dt


## hdfs 사용 클라이언트 객체 생성
- hdfs 연결 담당
- read/write/append/update 등과 관련되 모듈 포함하고 있음
- InsecureClient("연결주소", user = "사용자)
- 파일 읽기
    - InsecureClient.read(파일명(파일경로 포함))
        - 해당 파일을 읽을 수 있는 reader 객체를 반환
    - InsecureClient.write(저장할 파일명(파일경로 포함))
        - 해당 파일을 읽을 수 있는 writer 객체를 반환
    - InsecureClient.write(저장할 파일명(파일경로 포함), "추가될 내용", append= True)
        - 기존파일에 추가될 내용을 append() : 파일 끝에 쓰기
     
    - InsecureClient.upload(hdfs에 복사될 파일명(경로포함), 원본 폴더명(경로포함))
        - 지정된 원본 폴더 파일을 저장할 hdfd 폴더에 복사
        - 폴더를 생성하고 시작 : 기존에 동일 이름의 폴더가 있으면 에러 발생

In [4]:
# 9870port : name node 연결 port 
# 9000port : data node(hdfs) 연결 port
client = InsecureClient("http://localhost:9870", user="root")

## hdfs로 부터 읽기
- 파일과 연결(open)
- 읽기
- 파일 연결 끊기(close) : 개발자가 close해줘야 자원 정리됨 // 까먹거나 그런경우 자주 발생
    - with문을 사용하면 자동으로 open ~ close가 진행됨

In [5]:
# 파일 읽기 위한 reader 객체 생성 후 진행
with client.read("/rdd/score.txt") as reader :
    score = reader.read()

In [6]:
# byte code 읽어옴
score

b'\xea\xb9\x80\xec\xb2\xa0\xec\x88\x98 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 50\r\n\xed\x99\x8d\xea\xb8\xb8\xeb\x8f\x99 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 80\r\n\xec\x9e\x84\xea\xba\xbd\xec\xa0\x95 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 60\r\n\xec\x9e\x84\xec\x9a\x94\xed\x99\x98 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 100\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xec\x9d\xb4\xec\x9c\xa4\xec\x97\xb4 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 90\r\n\xec\xb5\x9c\xec\x97\xb0\xec\x84\xb1 \xec\x9e\xa5\xea\xb3\xa0 100\xec\xb5\x9c\xec\x97\xb0\xec\x84\xb1 \xec\x9e\xa5\xea\xb3\xa0 100\xec\xb5\x9c\xec\x97\xb0\xec\x84\xb1 \xec\x9e\xa5\xea\xb3\xa0 100\xec\xb5\x9c\xec\x97\xb0\xec\x84\xb1 \xec\x9e\xa5\xea\xb3\xa0 100'

In [7]:
# https://docs.python.org/ko/3/library/stdtypes.html#bytes-and-bytearray-operations
# byte code decode해서 저장 후 확인
# bytes.decode(data)
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90
최연성 장고 100최연성 장고 100최연성 장고 100최연성 장고 100


## hdfs에 쓰기
- hdd(note 디렉터리)에 잇는 data(file)를 hdfs로 복사
1. 로컬 데이터 파일 열기 : open() python method 사용, with 등록
2. hdfs client 통해서 쓰기 객체 with 등록
3. open 객체 이용해서 한줄씩 읽어서 -> hdfs에 쓰기
- 현재 위치 : /root/note/

In [8]:
with open("data/corona_data/sido_area.csv", encoding="CP949") as reader, client.write("/corona_data/sido_area_tmp.csv") as writer :
    for line in reader: # 읽어와서
        writer.write(line.encode("CP949")) # 원래 encoding 방식을 유지 시켜야함, 바로쓰기

HdfsError: /corona_data/sido_area_tmp.csv for client 127.0.0.1 already exists
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.startFile(FSDirWriteFileOp.java:388)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2819)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2713)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.create(NameNodeRpcServer.java:830)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.create(ClientNamenodeProtocolServerSideTranslatorPB.java:504)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:621)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:589)
	at org.apache.hadoop.ipc.ProtobufRpcEngi

### 인구 사회 데이터 DW에 저장
- 인구, 면적, 다중이용 시설 data

In [15]:
# 폴더 전송 : 폴더 내에 있는 파일을 hdfs 특정 디렉터리 모두 복수
# 해당 모듈은 디렉터리와 파일부터 생성하기 쓰기 진행
# 프로그램으로 파일 한줄씩 일어와서 바로 hdfs파일에 쓰기
# client.upload(data, source)
client.upload("/corona_data/tmp", "data/corona_data")

HdfsError: Remote path '/corona_data/tmp/corona_data' already exists.

## hdfs에 수정하기
- ***data 추가***이므로 data 형식과 encoding방식은 기존 파일과 동일하게 구성

In [16]:
client.write("/rdd/score.txt", "최연성 장고 100".encode("UTF-8"), append = True)

In [17]:
# 수정도니 파일 읽어와서 확인
# reader = client.read("/rdd/score.txt")와 같은 동작이므로 reader는 객체변수
with client.read("/rdd/score.txt") as reader :
    score = reader.read()
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90
최연성 장고 100최연성 장고 100최연성 장고 100최연성 장고 100


## hdfs 권한 수정
- 리눅스 명령어(chmod)
- InsecureClient 클래스의 set_permission('권한 변경할 파일(폴더 포함)', "권한문자")
- 권한문자 : 10진수 권한
    - ex. 777 : rwxrwxrwx -> 모든 user가 읽기/쓰기/실행 권한을 갖는다
    - tip. 777권한은 temp 폴더 등이 갖는 권한이고 공격대상이 될 수 있는 권한이므로 부여 시 주의 해야함

In [18]:
client.set_permission("/corona_data/tmp", "777")

## hdfs 삭제
- InsecureClient 클래스의 delete('삭제할 파일(경로포함)')
- 삭제가 정상적으로 진행되면 True 반환
- False가 반환된 경우 대부분 파일명 문제

In [19]:
client.delete("/corona_data/tmp/sido_area.xlsx")

False

## 실습 DataPipeline
- DataWarehouse
    - 현재 실행 중인 ubuntu server의 HDD -> hadoop의 FS로 변경된 공간을 사용 예정
        - hadoop 설치
        - hdfs 구성(format 등)
    - Extract 과정 거쳐서 hdfs에 데이터 저장됨
        1. corona 발생현황(일간 데이터) : api 수집
        2. corona vaccine 접종현황(일간 데이터) : 파일데이터에서 특정일에 대한 데이터 json형식으로 변환하여 수집
        3. 인구 사회 데이터 : 파일 데이터(변환 없이 사용)
- Transformation
    - HDFS에서 데이터 load해와서 spark 모듈이용 transformation 진행
    - DataWarehouse의 데이터보다 정형화된 데이터로 transform
- DataMart Load
    - transformation 진행된 data를 mysql db에 저장(적재)
- Transformation
    - DM에서 데이터 load해와서 spark 모듈 이용 transformation 진행
    - DM의 데이터보다 정형화되고 논리적 의미를 포함하는 데이터로 transform
- 운영DB에 load
    - transformation 진행된 data를 mysql db에 저장(적재)
    - 운영DB에 적재되는 데이터는 서비스와 연결됨
    - 정형화되고 논리적 의미를 포함해야함

## REST_API로 데이터를 호출해 HDFS에 저장
1. 데이터 신청
2. site에서 데이터 수집을 위한 url 제공
3. 가이드하고 잇는 방식(get/post)으로 데이터 요청 신행
    - header와 params 를 요구
        - 생성해서 요청시마다 사용
        - params에 service key를 포함시켜 구성
4. 요청된 데이터가 프로그램 변수에 저장되면 이 데이터를 정리해서 저장
5. 아래의 함수 executeRestApi() 요청하고 데이터 받아 변수 저장하고 함수를 호출한 곳에 데이터 반환(json형태의 text를 반환)

In [5]:
# 데이터 요청에 필요한 함수 인수 받아서
# 데이터 요청하고 전송된 데이터를 반환하는 함수
# 데이터 요청이 실패했을 경우 예외 발생시킴
# log 구성해서 예외 발생 시 log에 기록되도록 진행
def executeRestApi(method, url, headers, params):  
    # params, headers는 dict로 구성되어야함
    # raise Exception('응답코드 : 500')  # 예외 테스트
    # err_num = 10/0 # 예외 테스트
    if method == "get": # header가 필요없음 - 파라미터로 사용해야 된다면 보통 빈 {} 디렉터리로
        res = requests.get(url, params=params, headers=headers)
    else:
        res = requests.post(url, data=params, headers=headers)

    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))
       
    return res.text



### 기준일자 함수
- 요청의 data의 기준이 되는 yyyy-mm-dd 형식의 날짜 문자열을 생성하는 함수
- 오늘로부터 며칠 전 데이터를 요청할 날짜 생성
    - ex. 어제 date : 오늘부터 1일전
    - ex. 1년 전 data : 오늘부터 365일 전

In [6]:
dt.datetime.now() - dt.timedelta(365)

datetime.datetime(2024, 1, 21, 5, 44, 21, 123706)

In [7]:
# 오늘로부터 며칠 전인지 날짜를 전달 받아서 그날짜부터 생성
def cal_std_day(before_day):   
    x = dt.datetime.now() - dt.timedelta(before_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)
    return str(year)+"-"+str(month)+"-"+str(day)
cal_std_day(365*3)
cal_std_day(0)
cal_std_day(-1)

'2022-01-21'

'2025-01-20'

'2025-01-21'

### log 기록

#### logger
- log 기록하는 프로그램
- log : 프로그램을 실행하면서 나타나는 일련의 사건
(error, event 등 물리적으로 발생하는 내용이나 네트워크 송수신중 발생하는 많은 내용)
- 파이썬 패키지 logging 이용
-
- log는 외부 연결 시 오류 관련 확인을 위해서 사용자 정의를 기록해야함
- restApi는 외부 서버 : log 기록 필요성 있음
- 사용자 정의 log 생성
    1. 로그 기록/저장할 디렉터리 필요
    2. 로그 관련 패키지 import
    3. 모듈 활용 객체 (== logger)생성 (logging.getLogger('로거 이름')
    4. 로그 파일(을 컨트롤하는) 핸들러 생성 (logging.FileHandler('로그파일명.log'))
    5. 로그 파일 핸들러를 로거에 추가(로거와 로그 파일을 연결) : logger.addHandler(로그 파일 핸들러 객체명)

In [8]:
import logging

In [9]:
co_logger = logging.getLogger("corona_api")
f_handler = logging.FileHandler("./log/rest_api/"+cal_std_day(0)+".log")
co_logger.addHandler(f_handler)

In [10]:
# 이렇게 문자열로 찍을 수도 있음
# json양식으로 log를 저장하면 다음에 읽어와서 처리하기도 편하다.
# 강제 error 발생시켜서 log 기록 확인
# 실제 프로그램에서는 예외 또는 에러가 발생할 때 기록을 위한 logger error 발생
co_logger.error("corona_patient_"+cal_std_day(0)+".json 다운로드 실패")
# 발생된 에러의 log는 내가 만든 /root/note/log/rest_api 디렉터리 log 파일에서 확인

### api 호출

In [11]:
url = "http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api"
service_key = "ry3wiibrfwQzUSJCd7jeqtPqKmdasnJANipYjN3PWE6QgWbq9R+s1oKOofHF3go52uYT9jX7HCvCu3W0Yk8reA=="
# 요청하여 응답받은 rest_api data를 hdfs에 저장할 dir
file_dir = "/corona_data/patient/"

- 기본함수
    - executeRestApi(method, url, headers, params)
        - 대부분의 요청에 적용되는 함수
    - params는 여러 data를 설정해야함
        - 파라미터 생성함수 : create_param(std_day)
            - std_day : 날짜가 아닌 오늘부터 얼마나 이전 데이터를 요청할 것인지에 대한 수치

In [12]:
def create_param(std_day):
    return {
        "serviceKey":service_key,
        "pageNo":1,
        "numOfRows":"500",
        "apiType":"JSON",
        "std_day":cal_std_day(std_day)
    }
create_param(365)

{'serviceKey': 'ry3wiibrfwQzUSJCd7jeqtPqKmdasnJANipYjN3PWE6QgWbq9R+s1oKOofHF3go52uYT9jX7HCvCu3W0Yk8reA==',
 'pageNo': 1,
 'numOfRows': '500',
 'apiType': 'JSON',
 'std_day': '2024-01-21'}

In [13]:
params = create_param(365*3)
# log 기록할 내용 결정(개발자가 결정)
# json형식(dict)으로 log 기록하면 분석이 쉬워짐
log_dic={
    "is_success":"Fail",
    "type":"corona_patient",
    "std_day":params["std_day"],
    "params":params
}

- 날짜별 발생현황 파일 요청
    - 해당 파일에 날짜를 파일명에 포함시켜서
    - 정리없이 hdfs 저장

In [22]:
# 요청해서 json 응답을 받고, hdfs에 저장하는 작업
try :
    res = executeRestApi("get", url, {}, params) # api에 data 요청 후 결과 응답
    file_name = "corona_patient_"+cal_std_day(365*3)+".json" # hdfs에 저장할 파일명 생성
    client.write(file_dir+file_name, res, encoding = "utf-8") # hdfs에 저장
except Exception as e: # 일반적인 python 예외
    log_dic["err_msg"] = e.__str__()
    log_json = json.dumps(log_dic, ensure_ascii=False) # log json 형태로 기록
    co_logger.error(log_json) # logger가 발생시키는 강제 에러 // 예외가 발생하면 로거입장에서도 에러를 발생시켜서 기록을 하겠다.

In [23]:
# print(res)

In [24]:
create_param(365*3+6)["std_day"]

'2022-01-15'

In [25]:
for i in range(365*3, 365*3+6) :
    params = create_param(i)
    print(params["std_day"])
    try :
        res = executeRestApi("get", url, {}, params) 
        file_name = "corona_patient_"+cal_std_day(i)+".json"
        client.write(file_dir+file_name, res, encoding = "utf-8")
    except Exception as e:
        log_dic["err_msg"] = e.__str__()
        log_json = json.dumps(log_dic, ensure_ascii=False) 
        co_logger.error(log_json)

2022-01-21
2022-01-20
2022-01-19
2022-01-18
2022-01-17
2022-01-16


- 2022-01-13~2022-01-18 코로나 발생 현황 data를 hdfs에 json 형식으로 저장

## corona_vaccine 접종 현황 data
- 파일로 제공
- 파일 data 읽어서 json 형식으로 변경한 후에 hdfs에 저장
- 필요 data 추출 후 정리 형식 변환

In [14]:
import pandas as pd

In [15]:
pop = pd.read_csv("./data/corona_data/sido_population.csv", encoding="cp949")

vac_v1 = pd.read_csv("./data/corona_data/Covid19Vaccine.csv", index_col=0)
vac_v1.head(1)

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-11-09 00:00:00,제주특별자치도,6.0,5.0,591064,585442.0,591058,585437.0,10176,38.0,439144.0,439106.0


- 백신 데이터 광역시도명이 기 수집한 데이터의 광역시도명과 다르기 때문에 백신데이터를 변경 후 파일 저장

In [16]:
vac_v1["sido"]
vac_v1.iloc[1]

baseDate
2022-11-09 00:00:00    제주특별자치도
2022-11-09 00:00:00         전국
2022-11-09 00:00:00      서울특별시
2022-11-09 00:00:00      부산광역시
2022-11-09 00:00:00      대구광역시
                        ...   
2021-03-11 00:00:00       경상북도
2021-03-11 00:00:00       경상남도
2021-03-11 00:00:00    제주특별자치도
2021-03-11 00:00:00         전국
2021-03-11 00:00:00        강원도
Name: sido, Length: 10158, dtype: object

sido                            전국
firstCnt                     175.0
secondCnt                    224.0
totalFirstCnt             45123225
totalSecondCnt          44693321.0
accumulatedFirstCnt       45123050
accumulatedSecondCnt    44693097.0
id                           10159
thirdCnt                    1678.0
totalThirdCnt           33673146.0
accumulatedThirdCnt     33671468.0
Name: 2022-11-09 00:00:00, dtype: object

In [17]:
# 원본 데이터 추출 : 중복 없이 추출
vac_v1[["sido"]].drop_duplicates()["sido"]

baseDate
2022-11-09 00:00:00    제주특별자치도
2022-11-09 00:00:00         전국
2022-11-09 00:00:00      서울특별시
2022-11-09 00:00:00      부산광역시
2022-11-09 00:00:00      대구광역시
2022-11-09 00:00:00      광주광역시
2022-11-09 00:00:00      인천광역시
2022-11-09 00:00:00      대전광역시
2022-11-09 00:00:00        경기도
2022-11-09 00:00:00    세종특별자치시
2022-11-09 00:00:00       경상남도
2022-11-09 00:00:00       경상북도
2022-11-09 00:00:00       전라남도
2022-11-09 00:00:00       전라북도
2022-11-09 00:00:00       충청남도
2022-11-09 00:00:00       충청북도
2022-11-09 00:00:00        강원도
2022-11-09 00:00:00      울산광역시
2021-04-26 00:00:00         기타
Name: sido, dtype: object

In [18]:
before = [sido for sido in vac_v1[["sido"]].drop_duplicates()["sido"]]
before.sort()
before.remove("기타")
print(before)

['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전국', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [19]:
pop["loc"]

0     전국
1     서울
2     부산
3     대구
4     인천
5     광주
6     대전
7     울산
8     세종
9     경기
10    강원
11    충북
12    충남
13    전북
14    전남
15    경북
16    경남
17    제주
Name: loc, dtype: object

In [20]:
after = [sido for sido in pop["loc"]]
after.sort()
print(after)

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전국', '전남', '전북', '제주', '충남', '충북']


In [21]:
vac_v1["sido"].replace(before, after)

baseDate
2022-11-09 00:00:00    제주
2022-11-09 00:00:00    전국
2022-11-09 00:00:00    서울
2022-11-09 00:00:00    부산
2022-11-09 00:00:00    대구
                       ..
2021-03-11 00:00:00    경북
2021-03-11 00:00:00    경남
2021-03-11 00:00:00    제주
2021-03-11 00:00:00    전국
2021-03-11 00:00:00    강원
Name: sido, Length: 10158, dtype: object

In [22]:
for i in range(0, len(after)) :
    vac_v1["sido"] = vac_v1["sido"].replace(before[i], after[i])

In [23]:
# 2022-01-18에 해당하는 data를 vac_v1에서 추출
# "2022-11-09 00:00:00"
finVac = vac_v1.loc[vac_v1.index.str.contains("2022-01-18")]
finVac.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 2022-01-18 00:00:00 to 2022-01-18 00:00:00
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sido                  18 non-null     object 
 1   firstCnt              18 non-null     float64
 2   secondCnt             18 non-null     float64
 3   totalFirstCnt         18 non-null     int64  
 4   totalSecondCnt        18 non-null     float64
 5   accumulatedFirstCnt   18 non-null     int64  
 6   accumulatedSecondCnt  18 non-null     float64
 7   id                    18 non-null     int64  
 8   thirdCnt              18 non-null     float64
 9   totalThirdCnt         18 non-null     float64
 10  accumulatedThirdCnt   18 non-null     float64
dtypes: float64(7), int64(3), object(1)
memory usage: 1.7+ KB


## Json 구성

In [24]:
# 레코드 추출 - iloc[]
finVac.iloc[4]

sido                           대구
firstCnt                    893.0
secondCnt                  4524.0
totalFirstCnt             1997407
totalSecondCnt          1945813.0
accumulatedFirstCnt       1996514
accumulatedSecondCnt    1941289.0
id                           5644
thirdCnt                  15716.0
totalThirdCnt            978794.0
accumulatedThirdCnt      963078.0
Name: 2022-01-18 00:00:00, dtype: object

In [25]:
# hdfs에 저장할 데이터
cols = ["loc", "v1", "v2", "v3"]
data = []
for idx in range(0, len(finVac.index)):
    tuple_t = []
    tmp = finVac.iloc[idx]
    tuple_t.append(str(tmp.sido))
    tuple_t.append(str(tmp.totalFirstCnt))
    tuple_t.append(str(tmp.totalSecondCnt))
    tuple_t.append(str(tmp.totalThirdCnt))
    # print(tuple_t)
    data.append(dict(zip(cols, tuple_t)))

In [26]:
# 실제 data
data

[{'loc': '광주', 'v1': '1247764', 'v2': '1220758.0', 'v3': '682394.0'},
 {'loc': '울산', 'v1': '956895', 'v2': '933311.0', 'v3': '476692.0'},
 {'loc': '인천', 'v1': '2532911', 'v2': '2484196.0', 'v3': '1330273.0'},
 {'loc': '대전', 'v1': '1231435', 'v2': '1203724.0', 'v3': '628469.0'},
 {'loc': '대구', 'v1': '1997407', 'v2': '1945813.0', 'v3': '978794.0'},
 {'loc': '부산', 'v1': '2849093', 'v2': '2788089.0', 'v3': '1554554.0'},
 {'loc': '서울', 'v1': '8239400', 'v2': '8090244.0', 'v3': '4304485.0'},
 {'loc': '전국', 'v1': '44505276', 'v2': '43582128.0', 'v3': '23741205.0'},
 {'loc': '세종', 'v1': '290455', 'v2': '282507.0', 'v3': '138100.0'},
 {'loc': '경기', 'v1': '11652853', 'v2': '11414202.0', 'v3': '5970505.0'},
 {'loc': '제주', 'v1': '580579', 'v2': '566811.0', 'v3': '295807.0'},
 {'loc': '경남', 'v1': '2848083', 'v2': '2783896.0', 'v3': '1517240.0'},
 {'loc': '경북', 'v1': '2260963', 'v2': '2202814.0', 'v3': '1247047.0'},
 {'loc': '전남', 'v1': '1632045', 'v2': '1602479.0', 'v3': '1031459.0'},
 {'loc': '전북'

In [27]:
# data를 설명하는 메타 데이터
res = {
    "meta" : {
        "desc":"지역별 코로나 예방접종 인구 현황",
        "cols":{
            "loc":"지역",
            "v1":"1차 접종 누적",
            "v2":"2차 접종 누적",
            "v3":"3차 접종 누적"
        },
        "stdDay":cal_std_day(365*3)
    },
    "data":data
}
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'v1': '1차 접종 누적', 'v2': '2차 접종 누적', 'v3': '3차 접종 누적'},
  'stdDay': '2022-01-21'},
 'data': [{'loc': '광주', 'v1': '1247764', 'v2': '1220758.0', 'v3': '682394.0'},
  {'loc': '울산', 'v1': '956895', 'v2': '933311.0', 'v3': '476692.0'},
  {'loc': '인천', 'v1': '2532911', 'v2': '2484196.0', 'v3': '1330273.0'},
  {'loc': '대전', 'v1': '1231435', 'v2': '1203724.0', 'v3': '628469.0'},
  {'loc': '대구', 'v1': '1997407', 'v2': '1945813.0', 'v3': '978794.0'},
  {'loc': '부산', 'v1': '2849093', 'v2': '2788089.0', 'v3': '1554554.0'},
  {'loc': '서울', 'v1': '8239400', 'v2': '8090244.0', 'v3': '4304485.0'},
  {'loc': '전국', 'v1': '44505276', 'v2': '43582128.0', 'v3': '23741205.0'},
  {'loc': '세종', 'v1': '290455', 'v2': '282507.0', 'v3': '138100.0'},
  {'loc': '경기', 'v1': '11652853', 'v2': '11414202.0', 'v3': '5970505.0'},
  {'loc': '제주', 'v1': '580579', 'v2': '566811.0', 'v3': '295807.0'},
  {'loc': '경남', 'v1': '2848083', 'v2': '2783896.0', 'v3': '15

In [29]:
file_dir_vac = "/corona_data/vaccine/"
file_name = "corona_vaccine_"+cal_std_day(365*3+3)+".json"
client.write(file_dir_vac+file_name, json.dumps(res, ensure_ascii=False),encoding="utf-8")

In [30]:
file_dir_vac = "/corona_data/vaccine/"
file_name = "corona_vaccine_"+cal_std_day(365*3+3)+".json"
vaccine = spark.read.json(file_dir_vac+file_name)
vaccine.show()

+----------------------+---------------------------+
|                  data|                       meta|
+----------------------+---------------------------+
|[{광주, 1247764, 12...|{{지역, 1차 접종 누적, 2...|
+----------------------+---------------------------+



In [36]:
# 파일 읽어오기
vac_v1 = pd.read_csv("data/corona_data/Covid19CaccineC2.csv", index_col = 0)

FileNotFoundError: [Errno 2] No such file or directory: 'data/corona_data/Covid19CaccineC2.csv'

In [ ]:
# 전체 코드
# hdfs에 저장할 데이터
for i in range(365*3, 365*3+6) :
    print(cal_std_day(i))
    cols = ["loc", "v1", "v2", "v3"]
    data = []
    finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(i))]
    for idx in range(0, len(finVac.index)):
        tuple_t = []
        tmp = finVac.iloc[idx]
        tuple_t.append(str(tmp.sido))
        tuple_t.append(str(tmp.totalFirstCnt))
        tuple_t.append(str(tmp.totalSecondCnt))
        tuple_t.append(str(tmp.thirdCnt))
        # print(tuple_t)
        data.append(dict(zip(cols, tuple_t)))
    
    # data를 설명하는 메타 데이터
    res = {
        "meta" : {
            "desc":"지역별 코로나 예방접종 인구 현황",
            "cols":{
                "loc":"지역",
                "v1":"1차 접종 누적",
                "v2":"2차 접종 누적",
                "v3":"3차 접종 누적"
            },
            "stdDay":cal_std_day(i)
        },
        "data":data
    }
    try :
        file_dir_vac = "/corona_data/vaccine/"
        file_name = "corona_vaccine_"+cal_std_day(i)+".json"
        client.write(file_dir_vac+file_name, json.dumps(res, ensure_ascii=False),encoding="utf-8")
    except Exception as e :
        log_dic = {
            "type":"/corona_data/vaccine/",
            "err_msg":e.__str__()
        }
        log_json = json.dumps(log_dic, ensure_ascii=False)
        co_logger.error(log_json)